# 項目：評估和清洗英國電燒公司銷售數據

## 分析目標

此數據分析的目的是，根據市場銷售數據，挖掘暢銷產品，以便制定更有效的市場策略來提升營收。  


本項目的在於展現評估數據乾淨與整潔度，並且基於評估結果，對數據進行清洗，從而得到可供下一步分析的數據。

## 簡介

原始資料集記錄了一家英國線上零售公司在 2010 年 12 月 1 日至 2011 年 12 月 9 日期間的所有交易情況，涵蓋了該公司在全球不同國家和地區的業務資料。該公司主要銷售涵蓋各個場景的禮品，包括但不限於生日禮品、婚禮紀念品、聖誕禮品等等。該公司的客戶群主要包括批發商和個人消費者，其中批發商佔了相當大的比例。

資料欄位的含義如下：
- `InvoiceNo`：發票號碼。6 位數，作為交易的唯一識別碼。如果這個代碼以字母"c"開頭，表示這筆交易被取消。
- `StockCode`：產品代碼。5 位數，作為產品的唯一識別碼。
- `Description`：產品名稱。
- `Quantity`：產品在交易中的數量。
- `InvoiceDate`：發票日期和時間。交易發生的日期和時間。
- `UnitPrice`：單價。價格單位為英鎊（£）。
- `CustomerID`：客戶編號。5 位數，作為客戶的唯一識別碼。
- `Country`：國家名稱。客戶所居住的國家的名稱。

## 讀取數據

導入數據分析所需的函式庫，從 postgres 資料庫中，取出所需的資料內容，並解析為 DataFrame，並賦值給變數`original_data`。

In [34]:
import pandas as pd
from src.util.postgres.dataframe_and_postgres import DataframeAndPostgres

In [35]:
dataframe_and_postgres = DataframeAndPostgres()
original_data = dataframe_and_postgres.dataframe_from_postgres("e_commerce")
original_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## 評估數據

在這一部分，我將對上一部分建立的`original_data`這個 DataFrame 所包含的資料進行評估。

評估主要從兩個方面進行，結構和內容，即整齊度和乾淨度：
1. 結構：資料的結構性問題指不符合"每列是一個變量，每行是一個觀察值，每個單元格是一個值"這三個標準。
2. 內容：數據的內容性問題包括存在丟失數據、重複數據、無效數據等。

### 評估數據的整齊度

In [36]:
original_data.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
230476,557150,21843,RED RETROSPOT CAKE STAND,2,6/17/2011 11:02,10.95,15555.0,United Kingdom
235143,557626,21498,RED RETROSPOT WRAP,25,6/21/2011 14:29,0.42,14175.0,United Kingdom
344033,566979,22086,PAPER CHAIN KIT 50'S CHRISTMAS,12,9/16/2011 9:54,2.95,13141.0,United Kingdom
297650,562955,23235,STORAGE TIN VINTAGE LEAF,1,8/11/2011 10:14,5.79,NaN,United Kingdom
15196,537638,21725,SWEETIES STICKERS,2,12/7/2010 15:28,1.66,NaN,United Kingdom
525249,580612,22424,ENAMEL BREAD BIN CREAM,1,12/5/2011 11:58,24.96,NaN,United Kingdom
470214,576617,23007,SPACEBOY BABY GIFT SET,1,11/15/2011 16:57,10.79,NaN,United Kingdom
168470,551056,20766,GARDEN PATH SKETCHBOOK,1,4/26/2011 11:51,3.75,16442.0,United Kingdom
179502,552278,82582,AREA PATROLLED METAL SIGN,2,5/8/2011 12:36,2.10,15059.0,United Kingdom
529627,580737,22951,60 CAKE CASES DOLLY GIRL DESIGN,24,12/6/2011 9:05,0.55,14140.0,United Kingdom


從抽樣的 10 行資料資料來看，資料符合"每列是一個變量，每行是一個觀察值，每個儲存格都是一個值"，具體來看 row 是關於某商品的一次交易，column 是交易相關的各個變量，因此不存在結構性問題。

### 評估數據的乾淨度

In [37]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


從輸出結果來看，資料共有 541909 個觀察值，而`Description`、`CustomerID`變數存在缺失值。

此外，`InvoiceDate`的資料型別應為日期，`CustomerID`的資料型別應為字串，應進行資料格式轉換。

#### 評估缺失值

在暸解`Description`存在缺失值後，根據條件提取出缺失觀察值。

In [38]:
original_data[original_data["Description"].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
190,537466,21488,None,-50,12/7/2010 10:33,0.0,NaN,United Kingdom
623,536414,22139,None,56,12/1/2010 11:52,0.0,NaN,United Kingdom
1396,537873,37461,None,1,12/8/2010 18:07,0.0,NaN,United Kingdom
1734,537874,85170A,None,1,12/8/2010 18:07,0.0,NaN,United Kingdom
1974,536545,21134,None,1,12/1/2010 14:32,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535322,581199,84581,None,-2,12/7/2011 18:26,0.0,NaN,United Kingdom
535326,581203,23406,None,15,12/7/2011 18:31,0.0,NaN,United Kingdom
535332,581209,21620,None,6,12/7/2011 18:35,0.0,NaN,United Kingdom
536981,581234,72817,None,27,12/8/2011 10:33,0.0,NaN,United Kingdom


有 1454 筆交易資料缺失`Description`變數值。

從輸出結果來看，這些缺失`Description`的交易數據，`UnitPrice`都是 0。為了驗證猜想，我們增加篩選條件，看是否存在`Description`變數缺失且`UnitPrice`不為 0 的資料。

In [39]:
original_data[(original_data["Description"].isnull()) & (original_data["UnitPrice"] != 0)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


篩選出來結果數為 0 條，說明缺失`Description`值的數據，同時也不具備有效的`UnitPrice`值。

`Description`表示產品名稱，`UnitPrice`表示產品單價，都是進行後續商品交易分析的重要變數。如果它們同時缺失/無效，我們認為資料無法提供有效含義，因此這些後續可以被刪除。

`CustomerID`變數同樣存在缺失值，因此也根據條件擷取缺失觀察值。

In [40]:
original_data[original_data["CustomerID"].isnull()]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
190,537466,21488,None,-50,12/7/2010 10:33,0.00,NaN,United Kingdom
623,536414,22139,None,56,12/1/2010 11:52,0.00,NaN,United Kingdom
948,537639,22734,amazon,30,12/7/2010 15:29,0.00,NaN,United Kingdom
1396,537873,37461,None,1,12/8/2010 18:07,0.00,NaN,United Kingdom
1446,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,12/1/2010 14:32,2.51,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
541536,581498,85099B,JUMBO BAG RED RETROSPOT,5,12/9/2011 10:26,4.13,NaN,United Kingdom
541537,581498,85099C,JUMBO BAG BAROQUE BLACK WHITE,4,12/9/2011 10:26,4.13,NaN,United Kingdom
541538,581498,85150,LADIES & GENTLEMEN METAL SIGN,1,12/9/2011 10:26,4.96,NaN,United Kingdom
541539,581498,85174,S/4 CACTI CANDLES,1,12/9/2011 10:26,10.79,NaN,United Kingdom


`CustomerID`表示顧客編號，並非分析暢銷商品的必要變數。而從輸出結果來看，有些`CustomerID`缺少的交易資料仍然有效，因此保留此變數為空的觀察值。

#### 評估重複數據

根據資料變數的意義來看，雖然`InvoiceNo`、`StockCode`和`CustomerID`都是唯一識別符，但一次交易可能包含多件商品，因此`InvoiceNo`可以存在重複，不同交易可以包含同件商品，因此`StockCode`可以重複，顧客可以進行多次交易或下單多個商品，因此`CustomerID`也可以存在重複。

那麼針對此資料集，我們無需評估重複資料。

#### 評估不一致數據

不一致資料可能存在於`Country`變數中，我們要查看是否存在多個不同值指涉同一國家的情況。

In [41]:
original_data["Country"].value_counts()

Country
United Kingdom          495266
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
China                      288
Singapore                  229
USA                        218
UK                         211
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United States               73
United Arab Emirates        68


從`Country`變數值來看，`"USA"`、`"United States"`皆在表示美國，`"United Kingdom"`、`"UK"`、`"U.K."`皆在表示英國，因此應該在清洗步驟對這些值進行統一，只保留一個指代值。

#### 評估無效或錯誤數據

可以透過 DataFrame 的`describe`方法，對數值統計資訊進行快速了解。

In [42]:
original_data.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


從輸出結果來看，`Quantity`和`UnitPrice`存在負數，會對後續數值分析產生影響。

因此，我們先篩選出`Quantity`數值為負數的觀察值，進一步評估其意義。

In [43]:
original_data[original_data["Quantity"] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,12/1/2010 9:41,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,12/1/2010 9:49,4.65,15311.0,United Kingdom
190,537466,21488,None,-50,12/7/2010 10:33,0.00,NaN,United Kingdom
236,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,12/1/2010 10:24,1.65,17548.0,United Kingdom
237,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540450,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,12/9/2011 9:57,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,12/9/2011 10:28,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,12/9/2011 11:57,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,12/9/2011 11:58,1.25,17315.0,United Kingdom


從篩選結果來看，`Quantity`變數為負數的觀察值，`InvoiceNo`似乎都以C開頭，表示訂單被取消。

但此猜測不一定準確，為了驗證猜想，我們增加篩選條件，看是否存在`Quantity`變數為負數且`InvoiceNo`不以C開頭的觀察值。

In [44]:
original_data[(original_data["Quantity"] < 0) & (original_data["InvoiceNo"].str[0] != "C")]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
190,537466,21488,None,-50,12/7/2010 10:33,0.0,NaN,United Kingdom
2410,536589,21777,None,-10,12/1/2010 16:50,0.0,NaN,United Kingdom
4355,536764,84952C,None,-38,12/2/2010 14:42,0.0,NaN,United Kingdom
4452,538247,79067,None,-145,12/10/2010 12:09,0.0,NaN,United Kingdom
5875,549136,21414,None,-3,4/6/2011 15:23,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535333,581210,23395,check,-26,12/7/2011 18:36,0.0,NaN,United Kingdom
535335,581212,22578,lost,-1050,12/7/2011 18:38,0.0,NaN,United Kingdom
535336,581213,22576,check,-30,12/7/2011 18:38,0.0,NaN,United Kingdom
536908,581226,23090,missing,-338,12/8/2011 9:56,0.0,NaN,United Kingdom


以上猜想被證實錯誤，因為還存在`Quantity`變數為負數且`InvoiceNo`不以C開頭的觀察值。

但根據上述輸出結果，這些篩選出的觀察值的`UnitPrice`觀察值均為0，因此增加`UnitPrice`的條件進行驗證。

In [45]:
original_data[(original_data["Quantity"] < 0) & (original_data["InvoiceNo"].str[0] != "C") & (original_data["UnitPrice"] != 0)]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


根據輸出結果，猜想得到驗證，當`Quantity`變數為負數的時候，觀察值滿足以下條件之一：
- `InvoiceNo`以C開頭，表示訂單被取消
- `UnitPrice`為 0，說明單價為 0 英鎊

這些交易數據均不是有效成交數據，不貢獻銷售，不在後續分析範圍內，因此我們將在數據清理步驟，將`Quantity`為負數的觀察值刪除。

接下來篩選出`UnitPrice`為負數的觀察值。

In [46]:
original_data[original_data["UnitPrice"] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299989,A563186,B,Adjust bad debt,1,8/12/2011 14:51,-11062.06,NaN,United Kingdom
299990,A563187,B,Adjust bad debt,1,8/12/2011 14:52,-11062.06,NaN,United Kingdom


從輸出結果來看，`UnitPrice`為負數的觀察值都是壞帳調賬，不屬於實際商品交易數據，因此也在數據清理步驟中也將其刪除。

## 清洗數據

根據前面評估部分所得到的結論，我們需要進行的資料清理包括：
- 把`InvoiceDate`變數的資料類型轉換為日期時間
- 把`CustomerID`變數的資料型別轉換為字串
- 把`Description`變數缺失的觀察值刪除
- 把`Country`變數值`"USA"`替換為`"United States"`
- 把`Country`變數值`"UK"`、`"U.K."`替換為`"United Kingdom"`
- 將`Quantity`變數值為負數的觀察值刪除
- 把`UnitPrice`變數值為負數的觀察值刪除

為了區分開經過清理的資料和原始的數據，我們建立新的變數`cleaned_data`，讓它為`original_data`複製出的副本。我們之後的清理步驟都會被運用在`cleaned_data`上。

In [47]:
cleaned_data = original_data.copy()
cleaned_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


把`InvoiceDate`變數的資料型別轉換為日期時間型別：

In [48]:
cleaned_data["InvoiceDate"] = pd.to_datetime(cleaned_data["InvoiceDate"])
cleaned_data["InvoiceDate"]

0        2010-12-01 08:26:00
1        2010-12-01 08:26:00
2        2010-12-01 08:26:00
3        2010-12-01 08:26:00
4        2010-12-01 08:26:00
                 ...        
541904   2011-12-09 12:50:00
541905   2011-12-09 12:50:00
541906   2011-12-09 12:50:00
541907   2011-12-09 12:50:00
541908   2011-12-09 12:50:00
Name: InvoiceDate, Length: 541909, dtype: datetime64[ns]

把`CustomerID`變數的資料型別轉換為字串：

In [49]:
cleaned_data["CustomerID"] = cleaned_data["CustomerID"].astype(str)
cleaned_data["CustomerID"]

0         17850.0
1         17850.0
2         17850.0
3         17850.0
4         17850.0
           ...   
541904    12680.0
541905    12680.0
541906    12680.0
541907    12680.0
541908    12680.0
Name: CustomerID, Length: 541909, dtype: object

把`CustomerID`變數值結尾的`".0"`刪除：

In [50]:
cleaned_data["CustomerID"] = cleaned_data["CustomerID"].str.slice(0, -2)
cleaned_data["CustomerID"]

0         17850
1         17850
2         17850
3         17850
4         17850
          ...  
541904    12680
541905    12680
541906    12680
541907    12680
541908    12680
Name: CustomerID, Length: 541909, dtype: object

把`Description`變數缺少的觀察值刪除，並查看刪除後該列空缺值個數和：

In [51]:
cleaned_data.dropna(subset=["Description"], inplace=True)
cleaned_data["Description"].isnull().sum()

0

把`Country`變數值`"USA"`替換為`"United States"`，並檢查替換後`"USA"`變數值個數：

In [52]:
cleaned_data["Country"] = cleaned_data["Country"].replace({"USA": "United States"})
len(cleaned_data[cleaned_data["Country"] == "USA"])

0

把`Country`變數值`"UK"`、`"U.K."`替換為`"United Kingdom"`，並檢查替換後`"UK"`和`"U.K."`變數值個數：

In [53]:
cleaned_data["Country"] = cleaned_data["Country"].replace({"UK": "United Kingdom", "U.K.": "United Kingdom"})
print(len(cleaned_data[cleaned_data["Country"] == "UK"]))
print(len(cleaned_data[cleaned_data["Country"] == "U.K."]))

0
0


將`Quantity`變數值為負數的觀察值刪除，並檢查替換後`Quantity`變數值為負數的個數：

In [54]:
cleaned_data = cleaned_data[cleaned_data["Quantity"] >= 0]
len(cleaned_data[cleaned_data["Quantity"] < 0])

0

將`UnitPrice`變數值為負數的觀察值刪除，並檢查替換後`UnitPrice`變數值為負數的個數：

In [55]:
cleaned_data = cleaned_data[cleaned_data["UnitPrice"] >= 0]
len(cleaned_data[cleaned_data["UnitPrice"] < 0])

0

## 保存清洗後的資料

完成資料清理後，把乾淨整齊的資料寫回資料庫，表名稱為`e_commerce_cleaned`。

In [56]:
dataframe_and_postgres.dataframe_to_postgres("e_commerce_cleaned", cleaned_data)